In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.preprocessing import OneHotEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import LabelEncoder

import joblib

In [2]:
df = pd.read_csv('mxmh_survey_results.csv')
df = df.rename({'Primary streaming service':'PrimaryStreamingService', 
                'Hours per day': 'HoursPerDay', 
                'While working': 'WhileWorking', 
                'Fav genre': 'FavGenre', 
                'Foreign languages': 'ForeignLanguages', 
                'Frequency [Classical]': 'FrequencyClassical',
                'Frequency [Country]': 'FrequencyCountry',
                'Frequency [EDM]': 'FrequencyEDM',
                'Frequency [Folk]': 'FrequencyFolk',
                'Frequency [Gospel]': 'FrequencyGospel',
                'Frequency [Hip hop]': 'FrequencyHipHop',
                'Frequency [Jazz]': 'FrequencyJazz',
                'Frequency [K pop]': 'FrequencyKPop',
                'Frequency [Latin]': 'FrequencyLatin',
                'Frequency [Lofi]': 'FrequencyLofi',
                'Frequency [Metal]': 'FrequencyMetal',
                'Frequency [Pop]': 'FrequencyPop',
                'Frequency [R&B]': 'FrequencyRNB',
                'Frequency [Rap]': 'FrequencyRap',
                'Frequency [Rock]': 'FrequencyRock',
                'Frequency [Video game music]': 'FrequencyVideoGameMusic',
                'Music effects': 'MusicEffects'}, axis=1)
df = df.dropna(subset=['MusicEffects', 'Age', 'PrimaryStreamingService', 'WhileWorking', 'Instrumentalist', 'ForeignLanguages'])
df = df.dropna(axis=1)
df = df.drop(['Timestamp', 'Permissions', 'MusicEffects', 'PrimaryStreamingService'], axis=1)

for i in df.columns:
    df[i] = df[i].apply(lambda x: 1 if x == 'Yes' else x)
    df[i] = df[i].apply(lambda x: 0 if x == 'No' else x)

enc = OneHotEncoder(handle_unknown='ignore')


y = df[['FrequencyClassical', 'FrequencyCountry', 
        'FrequencyEDM', 'FrequencyFolk', 
        'FrequencyGospel', 'FrequencyHipHop',
        'FrequencyJazz', 'FrequencyKPop', 
        'FrequencyLatin', 'FrequencyLofi',
        'FrequencyMetal', 'FrequencyPop',
        'FrequencyRNB', 'FrequencyRap',
        'FrequencyRock', 'FrequencyVideoGameMusic']]



X_sub = list(zip(df.FavGenre))
enc.fit(X_sub)
feature_vectors = enc.transform(X_sub).toarray()


X = df[['Age', 'HoursPerDay', 'WhileWorking','Instrumentalist','Composer', 'Exploratory', 'ForeignLanguages', 'Anxiety', 'Depression', 'Insomnia', 'OCD']]
# X = df[['Age', 'HoursPerDay', 'Anxiety', 'Depression', 'Insomnia', 'OCD']]
X[['Classical', 'Country', 'EDM', 'Folk', 'Gospel', 'HipHop', 'Jazz',
       'KPop', 'Latin', 'Lofi', 'Metal', 'Pop', 'RNB', 'Rap', 'Rock',
       'VideoGameMusic']] = feature_vectors

display(X)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


model = RandomForestClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

y_pred = pd.DataFrame(y_pred, columns=['FrequencyClassical', 'FrequencyCountry', 
        'FrequencyEDM', 'FrequencyFolk', 
        'FrequencyGospel', 'FrequencyHipHop',
        'FrequencyJazz', 'FrequencyKPop', 
        'FrequencyLatin', 'FrequencyLofi',
        'FrequencyMetal', 'FrequencyPop',
        'FrequencyRNB', 'FrequencyRap',
        'FrequencyRock', 'FrequencyVideoGameMusic'])

label_encoder = LabelEncoder()
y_test_encoded = y_test.apply(label_encoder.fit_transform)
y_pred_encoded = y_pred.apply(label_encoder.transform)
# print(y_test)
# print(y_test_encoded)

f1_scores = []
precision_scores = []
recall_scores = []

for column in y_test.columns:
    
    p,r,f,s = precision_recall_fscore_support(y_test_encoded[column], y_pred_encoded[column], average=None, labels=[0, 1, 2, 3])
    print(column)
    print("F1:", f,"Precision:", p,"Recall:", r,"Support:", s)
    
    f1_scores.append(f)
    precision_scores.append(p)
    recall_scores.append(r)

# very frequently = 3, sometimes = 2, rarely = 1, never = 0


C:\Users\emily\AppData\Local\Temp\ipykernel_27096\1111414042.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[['Classical', 'Country', 'EDM', 'Folk', 'Gospel', 'HipHop', 'Jazz',
C:\Users\emily\AppData\Local\Temp\ipykernel_27096\1111414042.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[['Classical', 'Country', 'EDM', 'Folk', 'Gospel', 'HipHop', 'Jazz',
C:\Users\emily\AppData\Local\Temp\ipykernel_27096\1111414042.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

,Age,HoursPerDay,WhileWorking,Instrumentalist,Composer,Exploratory,ForeignLanguages,Anxiety,Depression,Insomnia,...,Jazz,KPop,Latin,Lofi,Metal,Pop,RNB,Rap,Rock,VideoGameMusic
2,18.0,4.0,0,0,0,0,1,7.0,7.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,61.0,2.5,1,0,1,1,1,9.0,7.0,3.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,18.0,4.0,1,0,0,1,0,7.0,2.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5,18.0,5.0,1,1,1,1,1,8.0,8.0,7.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,18.0,3.0,1,1,0,1,1,4.0,8.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731,17.0,2.0,1,1,0,1,1,7.0,6.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
732,18.0,1.0,1,1,0,1,1,3.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
733,19.0,6.0,1,0,1,1,0,2.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
734,19.0,5.0,1,1,0,0,0,2.0,3.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


FrequencyClassical
F1: [0.33802817 0.47540984 0.31746032 0.625     ] Precision: [0.38709677 0.453125   0.27027027 0.83333333] Recall: [0.3        0.5        0.38461538 0.5       ] Support: [40 58 26 20]
FrequencyCountry
F1: [0.63291139 0.35955056 0.         0.77777778] Precision: [0.53763441 0.3902439  0.         1.        ] Recall: [0.76923077 0.33333333 0.         0.63636364] Support: [65 48 20 11]
FrequencyEDM
F1: [0.52413793 0.19354839 0.30769231 0.68965517] Precision: [0.39175258 0.33333333 0.44444444 0.90909091] Recall: [0.79166667 0.13636364 0.23529412 0.55555556] Support: [48 44 34 18]
FrequencyFolk
F1: [0.59541985 0.27160494 0.36734694 0.37037037] Precision: [0.52702703 0.23913043 0.5        0.83333333] Recall: [0.68421053 0.31428571 0.29032258 0.23809524] Support: [57 35 31 21]
FrequencyGospel
F1: [0.8 0.  0.  0. ] Precision: [0.67132867 0.         0.         0.        ] Recall: [0.98969072 0.         0.         0.        ] Support: [97 37  9  1]
FrequencyHipHop
F1: [0.411764

c:\Users\emily\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [3]:
def evaluate(X_train, X_test, y_train):
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    joblib.dump(model, "music_predictor.joblib")
    y_pred = model.predict(X_test)
    y_pred = pd.DataFrame(y_pred, columns=['FrequencyClassical', 'FrequencyCountry', 
        'FrequencyEDM', 'FrequencyFolk', 
        'FrequencyGospel', 'FrequencyHipHop',
        'FrequencyJazz', 'FrequencyKPop', 
        'FrequencyLatin', 'FrequencyLofi',
        'FrequencyMetal', 'FrequencyPop',
        'FrequencyRNB', 'FrequencyRap',
        'FrequencyRock', 'FrequencyVideoGameMusic'])
    print("---------Predicted----------")
    display(y_pred)


In [4]:
# original
df = pd.read_csv('mxmh_survey_results.csv')
df = df.rename({'Primary streaming service':'PrimaryStreamingService', 
                'Hours per day': 'HoursPerDay', 
                'While working': 'WhileWorking', 
                'Fav genre': 'FavGenre', 
                'Foreign languages': 'ForeignLanguages', 
                'Frequency [Classical]': 'FrequencyClassical',
                'Frequency [Country]': 'FrequencyCountry',
                'Frequency [EDM]': 'FrequencyEDM',
                'Frequency [Folk]': 'FrequencyFolk',
                'Frequency [Gospel]': 'FrequencyGospel',
                'Frequency [Hip hop]': 'FrequencyHipHop',
                'Frequency [Jazz]': 'FrequencyJazz',
                'Frequency [K pop]': 'FrequencyKPop',
                'Frequency [Latin]': 'FrequencyLatin',
                'Frequency [Lofi]': 'FrequencyLofi',
                'Frequency [Metal]': 'FrequencyMetal',
                'Frequency [Pop]': 'FrequencyPop',
                'Frequency [R&B]': 'FrequencyRNB',
                'Frequency [Rap]': 'FrequencyRap',
                'Frequency [Rock]': 'FrequencyRock',
                'Frequency [Video game music]': 'FrequencyVideoGameMusic',
                'Music effects': 'MusicEffects'}, axis=1)
df = df.dropna(subset=['MusicEffects', 'Age', 'PrimaryStreamingService', 'WhileWorking', 'Instrumentalist', 'ForeignLanguages'])
df = df.dropna(axis=1)
df = df.drop(['Timestamp', 'Permissions', 'MusicEffects', 'PrimaryStreamingService'], axis=1)

for i in df.columns:
    df[i] = df[i].apply(lambda x: 1 if x == 'Yes' else x)
    df[i] = df[i].apply(lambda x: 0 if x == 'No' else x)

enc = OneHotEncoder(handle_unknown='ignore')


y = df[['FrequencyClassical', 'FrequencyCountry', 
        'FrequencyEDM', 'FrequencyFolk', 
        'FrequencyGospel', 'FrequencyHipHop',
        'FrequencyJazz', 'FrequencyKPop', 
        'FrequencyLatin', 'FrequencyLofi',
        'FrequencyMetal', 'FrequencyPop',
        'FrequencyRNB', 'FrequencyRap',
        'FrequencyRock', 'FrequencyVideoGameMusic']]



X_sub = list(zip(df.FavGenre))
enc.fit(X_sub)
feature_vectors = enc.transform(X_sub).toarray()


X = df[['Age', 'HoursPerDay', 'WhileWorking','Instrumentalist','Composer', 'Exploratory', 'ForeignLanguages', 'Anxiety', 'Depression', 'Insomnia', 'OCD']]

X[['Classical', 'Country', 'EDM', 'Folk', 'Gospel', 'HipHop', 'Jazz',
       'KPop', 'Latin', 'Lofi', 'Metal', 'Pop', 'RNB', 'Rap', 'Rock',
       'VideoGameMusic']] = feature_vectors
display(X)
display(y)


# Test
challenge = pd.DataFrame(data={'Age': [18.0], 'HoursPerDay': [3.0], 'WhileWorking': [1],'Instrumentalist':[1],'Composer':[0], 'Exploratory':[1], 'ForeignLanguages':[1], 'Anxiety': [4.0], 'Depression': [4.0], 'Insomnia': [3.0], 'OCD': [5.0], 'Classical': [0.0], 'Country': [0.0], 'EDM': [0.0], 'Folk': [0.0], 'Gospel': [0.0], 'HipHop': [1.0], 'Jazz': [0.0],
       'KPop': [0.0], 'Latin': [0.0], 'Lofi': [0.0], 'Metal': [0.0], 'Pop': [0.0], 'RNB': [0.0], 'Rap': [0.0], 'Rock': [0.0],
       'VideoGameMusic': [0.0]})

display(challenge)
evaluate(X, challenge, y)

C:\Users\emily\AppData\Local\Temp\ipykernel_27096\1516773097.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[['Classical', 'Country', 'EDM', 'Folk', 'Gospel', 'HipHop', 'Jazz',
C:\Users\emily\AppData\Local\Temp\ipykernel_27096\1516773097.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[['Classical', 'Country', 'EDM', 'Folk', 'Gospel', 'HipHop', 'Jazz',
C:\Users\emily\AppData\Local\Temp\ipykernel_27096\1516773097.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

,Age,HoursPerDay,WhileWorking,Instrumentalist,Composer,Exploratory,ForeignLanguages,Anxiety,Depression,Insomnia,...,Jazz,KPop,Latin,Lofi,Metal,Pop,RNB,Rap,Rock,VideoGameMusic
2,18.0,4.0,0,0,0,0,1,7.0,7.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,61.0,2.5,1,0,1,1,1,9.0,7.0,3.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,18.0,4.0,1,0,0,1,0,7.0,2.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5,18.0,5.0,1,1,1,1,1,8.0,8.0,7.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,18.0,3.0,1,1,0,1,1,4.0,8.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731,17.0,2.0,1,1,0,1,1,7.0,6.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
732,18.0,1.0,1,1,0,1,1,3.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
733,19.0,6.0,1,0,1,1,0,2.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
734,19.0,5.0,1,1,0,0,0,2.0,3.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,FrequencyClassical,FrequencyCountry,FrequencyEDM,FrequencyFolk,FrequencyGospel,FrequencyHipHop,FrequencyJazz,FrequencyKPop,FrequencyLatin,FrequencyLofi,FrequencyMetal,FrequencyPop,FrequencyRNB,FrequencyRap,FrequencyRock,FrequencyVideoGameMusic
2,Never,Never,Very frequently,Never,Never,Rarely,Rarely,Very frequently,Never,Sometimes,Sometimes,Rarely,Never,Rarely,Rarely,Very frequently
3,Sometimes,Never,Never,Rarely,Sometimes,Never,Very frequently,Sometimes,Very frequently,Sometimes,Never,Sometimes,Sometimes,Never,Never,Never
4,Never,Never,Rarely,Never,Rarely,Very frequently,Never,Very frequently,Sometimes,Sometimes,Never,Sometimes,Very frequently,Very frequently,Never,Rarely
5,Rarely,Sometimes,Never,Never,Never,Sometimes,Very frequently,Very frequently,Rarely,Very frequently,Rarely,Very frequently,Very frequently,Very frequently,Very frequently,Never
6,Sometimes,Never,Rarely,Sometimes,Rarely,Rarely,Sometimes,Never,Rarely,Rarely,Rarely,Rarely,Rarely,Never,Never,Sometimes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731,Very frequently,Rarely,Never,Sometimes,Never,Sometimes,Rarely,Never,Sometimes,Rarely,Rarely,Very frequently,Never,Rarely,Very frequently,Never
732,Rarely,Rarely,Never,Never,Never,Never,Rarely,Never,Never,Rarely,Never,Very frequently,Never,Never,Sometimes,Sometimes
733,Rarely,Sometimes,Sometimes,Rarely,Rarely,Very frequently,Rarely,Rarely,Rarely,Sometimes,Rarely,Sometimes,Sometimes,Sometimes,Rarely,Rarely
734,Very frequently,Never,Never,Never,Never,Never,Rarely,Never,Never,Never,Never,Never,Never,Never,Never,Sometimes


,Age,HoursPerDay,WhileWorking,Instrumentalist,Composer,Exploratory,ForeignLanguages,Anxiety,Depression,Insomnia,...,Jazz,KPop,Latin,Lofi,Metal,Pop,RNB,Rap,Rock,VideoGameMusic
0,18.0,3.0,1,1,0,1,1,4.0,4.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


---------Predicted----------


,FrequencyClassical,FrequencyCountry,FrequencyEDM,FrequencyFolk,FrequencyGospel,FrequencyHipHop,FrequencyJazz,FrequencyKPop,FrequencyLatin,FrequencyLofi,FrequencyMetal,FrequencyPop,FrequencyRNB,FrequencyRap,FrequencyRock,FrequencyVideoGameMusic
0,Never,Never,Never,Never,Never,Very frequently,Never,Never,Never,Sometimes,Never,Sometimes,Very frequently,Very frequently,Very frequently,Rarely
